In [1]:
!gdown 1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
!gdown 1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1

Downloading...
From: https://drive.google.com/uc?id=1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
To: /content/test_data.csv
100% 565k/565k [00:00<00:00, 32.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1
To: /content/train_data.csv
100% 2.24M/2.24M [00:00<00:00, 55.0MB/s]


In [2]:
import numpy as np
import pandas as pd
import gensim
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [3]:
train_df = pd.read_csv("train_data.csv")
train_df = train_df[train_df['confidence'] >= 0.8].reset_index(drop=True)

In [4]:
test_df = pd.read_csv("test_data.csv")
test_df = test_df[test_df['confidence'] >= 0.8].reset_index(drop=True)

In [5]:
print(train_df.shape, test_df.shape)

(1832, 111) (465, 111)


In [6]:
X_train = train_df.drop(['target'], axis=1)
y_train = train_df['target'].to_numpy()
X_test = test_df.drop(['target'], axis=1)
y_test = test_df['target'].to_numpy()

# Word2Vec

In [7]:
model_w2v = gensim.models.Word2Vec(window=10, min_count=2, vector_size=300)

In [8]:
texts = []
for text in X_train['clean_text']:
  texts.append(text.split())
for text in X_test:
  texts.append(text.split())

In [9]:
model_w2v.build_vocab(texts)

In [10]:
model_w2v.train(texts, total_examples=model_w2v.corpus_count, epochs=20)

(1162162, 1385380)

In [11]:
model_w2v.wv.most_similar("good")

[('guy', 0.8878058791160583),
 ('boyfriend', 0.8600823879241943),
 ('intention', 0.8520587682723999),
 ('obnoxious', 0.8463851809501648),
 ('tldr', 0.8433341979980469),
 ('mia', 0.840312659740448),
 ('enjoy', 0.833155632019043),
 ('friend', 0.829447865486145),
 ('nice', 0.8281909227371216),
 ('elementary', 0.8266186118125916)]

In [12]:
model_w2v.wv.doesnt_match(['good', 'great', 'nice', 'bad'])

'bad'

In [13]:
model_w2v.save('word2vec.model')

In [14]:
def document_vector(doc):
    voc = set(model_w2v.wv.index_to_key)
    doc = doc.split()
    doc = [word for word in doc if word in voc]
    return np.mean(model_w2v.wv[doc], axis=0)

In [15]:
train_w2v = []
for review in X_train['clean_text']:
  train_w2v.append(document_vector(review))

In [16]:
train_w2v = np.array(train_w2v)
train_w2v.shape

(1832, 300)

In [17]:
test_w2v = []
for review in X_test['clean_text']:
  test_w2v.append(document_vector(review))

In [18]:
test_w2v = np.array(test_w2v)

In [19]:
X_train_w2v= X_train.drop(['clean_text'], axis=1).to_numpy()
X_test_w2v = X_test.drop(['clean_text'], axis=1).to_numpy()

In [20]:
X_train_w2v = np.concatenate((X_train_w2v, train_w2v), axis=1)
X_test_w2v = np.concatenate((X_test_w2v, test_w2v), axis=1)

In [21]:
print(X_train_w2v.shape)
print(X_test_w2v.shape)

(1832, 409)
(465, 409)


# Training and Testing

In [22]:
def training_and_prediction(training_model, X_train, X_test, y_train, y_test):
  model = training_model
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  # print("Accuracy =", accuracy)
  # print("Confusion Matrix =\n", confusion_matrix(y_test, y_pred))
  # print(classification_report(y_test, y_pred))
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [23]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

## Naive Bayes

In [24]:
scaler = MinMaxScaler()
training_and_prediction(MultinomialNB(), scaler.fit_transform(X_train_w2v), scaler.fit_transform(X_test_w2v), y_train, y_test)

{'Accuracy': 0.7376344086021506,
 'Precision': 0.7006802721088435,
 'Recall': 0.8583333333333333,
 'F1-Score': 0.7715355805243446}

In [25]:
training_and_prediction(GaussianNB(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.6946236559139785,
 'Precision': 0.6737588652482269,
 'Recall': 0.7916666666666666,
 'F1-Score': 0.7279693486590039}

In [26]:
training_and_prediction(BernoulliNB(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7118279569892473,
 'Precision': 0.6906474820143885,
 'Recall': 0.8,
 'F1-Score': 0.7413127413127415}

## SVM

In [27]:
training_and_prediction(SVC(kernel='poly'), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7913978494623656,
 'Precision': 0.7739463601532567,
 'Recall': 0.8416666666666667,
 'F1-Score': 0.8063872255489023}

In [28]:
training_and_prediction(SVC(C=10), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.8,
 'Precision': 0.7859922178988327,
 'Recall': 0.8416666666666667,
 'F1-Score': 0.8128772635814889}

## Random Forest

In [29]:
training_and_prediction(RandomForestClassifier(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7462365591397849,
 'Precision': 0.744,
 'Recall': 0.775,
 'F1-Score': 0.7591836734693876}

## Logistic Regression

In [30]:
training_and_prediction(LogisticRegression(max_iter=5000, solver='saga'), X_train_w2v, X_test_w2v, y_train, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'Accuracy': 0.810752688172043,
 'Precision': 0.804,
 'Recall': 0.8375,
 'F1-Score': 0.8204081632653061}

## Gradient Boosting

In [31]:
training_and_prediction(XGBClassifier(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7913978494623656,
 'Precision': 0.7942386831275721,
 'Recall': 0.8041666666666667,
 'F1-Score': 0.7991718426501035}

## Deep Learning Techniques with Word2vec

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [33]:
X_train_w2v.shape[1]

409

### ANN

In [34]:
def ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate=0):
    model_dense = Sequential()
    model_dense.add(Dense(num_neurons_layer1, activation='relu', input_shape=(X_train_w2v.shape[1],)))
    model_dense.add(Dropout(dropout_rate))
    model_dense.add(Dense(num_neurons_layer2, activation='relu'))
    model_dense.add(Dense(1, activation='sigmoid'))
    model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_dense.fit(X_train_w2v, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_w2v, y_test))
    y_pred_dense = model_dense.predict(X_test_w2v)
    y_pred_dense = (y_pred_dense > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'num_neurons_layer2': num_neurons_layer2, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_dense)]

In [35]:
for num_neurons_layer1 in [128, 64]:
  for num_neurons_layer2 in [64, 32]:
    for dropout_rate in [0, 0.1, 0.2, 0.3, 0.4]:
      print(ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate))

15/15 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0}, {'Accuracy': 0.7935483870967742, 'Precision': 0.7706766917293233, 'Recall': 0.8541666666666666, 'F1-Score': 0.8102766798418972}]
15/15 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.1}, {'Accuracy': 0.7913978494623656, 'Precision': 0.8149779735682819, 'Recall': 0.7708333333333334, 'F1-Score': 0.7922912205567452}]
15/15 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.2}, {'Accuracy': 0.7870967741935484, 'Precision': 0.8051948051948052, 'Recall': 0.775, 'F1-Score': 0.7898089171974523}]
15/15 [==============================] - 0s 2ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.3}, {'Accuracy': 0.7978494623655914, 'Precision': 0.7874015748031497, 'Recall': 0.8333333333333334, 'F1-Score': 